# SPY vs Ticker comparison

## Getting list of tickers

In [3]:
import numpy as np
import yaml

# --- load config to get tickers order
with open("config.yaml", "r") as f:
    cfg = yaml.safe_load(f)

tickers = cfg["data"]["tickers"]  # adjust if your path differs
assert isinstance(tickers, list) and len(tickers) > 0
print(tickers)

['AAPL', 'MSFT', 'NVDA', 'AMZN', 'GOOGL', 'META', 'AVGO', 'AMD', 'INTC', 'TSLA', 'JNJ', 'PG', 'KO', 'PEP', 'JPM', 'BAC', 'XOM', 'CVX', 'CAT', 'HON', 'SPY']


## Loading prices

In [5]:
# --- load prices (flat vector aligned by ticker segments)
prices = np.load("data/prices.npy")  # shape (N,)

rows_per_ticker = 1238  # from your log
train_len = 866
val_len = 185
test_len = 187

# --- locate SPY in tickers list
spy_symbol = "SPY"
if spy_symbol not in tickers:
    raise ValueError(f"{spy_symbol} not found in config tickers list")

spy_idx = tickers.index(spy_symbol)

# --- extract SPY segment and test slice
start = spy_idx * rows_per_ticker
end = start + rows_per_ticker
spy_prices_seg = prices[start:end]

spy_test_prices = spy_prices_seg[train_len + val_len : train_len + val_len + test_len]
assert spy_test_prices.shape[0] == test_len

print(spy_test_prices)


[394.3396  393.30713 394.8413  395.24655 395.35278 393.74133 398.97122
 397.99667 399.42484 399.68533 399.61777 397.437   397.74582 398.16077
 391.84036 390.18073 397.94846 401.34503 400.93976 396.4335  393.71237
 390.92377 398.16077 398.26685 396.52036 398.37302 397.67822 397.15723
 398.52744 395.86417 400.66962 404.52924 403.94067 404.10477 399.5695
 396.67474 400.10986 405.29163 405.44595 403.19775 407.02847 412.9146
 412.12332 413.02072 411.5926  414.08212 414.82513 418.58838 421.34808
 421.84988 427.07977 425.62506 423.41687 421.24734 422.76797 419.57184
 417.85754 422.43872 422.65173 424.31757 429.3248  429.81873 429.17953
 425.81882 424.74368 425.81882 428.53067 431.97852 435.4071  435.13593
 436.64682 439.89136 440.8695  437.94458 437.94458 439.90103 441.102
 441.1698  438.2449  442.53543 443.37802 442.1093  435.9592  434.70978
 432.7437  436.5209  434.6226  431.71704 431.87198 431.62018 434.00272
 428.94708 425.79947 422.55487 422.75827 425.50882 424.35632 429.08267
 423.13602

## Util function

In [6]:
def compute_buyhold_equity(prices_1d: np.ndarray) -> float:
    p0 = float(prices_1d[0])
    p1 = float(prices_1d[-1])
    return p1 / max(p0, 1e-12)

In [7]:
spy_equity = compute_buyhold_equity(spy_test_prices)

print(f"[BENCH] SPY buy&hold test equity: {spy_equity:.4f}")

# plug your strategy test equity from GA output:
strategy_equity = 9.641518117197839
print(f"[BENCH] Strategy test equity:     {strategy_equity:.4f}")
print(f"[BENCH] Strategy / SPY multiple:  {strategy_equity / max(spy_equity, 1e-12):.4f}x")


[BENCH] SPY buy&hold test equity: 1.1763
[BENCH] Strategy test equity:     9.6415
[BENCH] Strategy / SPY multiple:  8.1968x
